# Optimizing Model Parameters

0. Prerequisite Code

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

1. Hyperparameters

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

2. Optimization Loop :

Loss Function

In [ ]:
loss_fn = nn.CrossEntropyLoss()

Optimizer

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

3. Full Implementation :

Train loop

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()
    for batch, (X, y) in enumerate(dataloader):

        pred = model(X)
        loss = loss_fn(pred, y)


        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Test loop

In [ ]:
def test_loop(dataloader, model, loss_fn):

    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0


    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310738  [   64/60000]
loss: 2.291548  [ 6464/60000]
loss: 2.272606  [12864/60000]
loss: 2.257719  [19264/60000]
loss: 2.250661  [25664/60000]
loss: 2.232831  [32064/60000]
loss: 2.224620  [38464/60000]
loss: 2.202049  [44864/60000]
loss: 2.206221  [51264/60000]
loss: 2.168969  [57664/60000]
Test Error: 
 Accuracy: 44.9%, Avg loss: 2.156715 

Epoch 2
-------------------------------
loss: 2.177720  [   64/60000]
loss: 2.156492  [ 6464/60000]
loss: 2.100369  [12864/60000]
loss: 2.105325  [19264/60000]
loss: 2.068060  [25664/60000]
loss: 2.020037  [32064/60000]
loss: 2.029950  [38464/60000]
loss: 1.957518  [44864/60000]
loss: 1.973156  [51264/60000]
loss: 1.896219  [57664/60000]
Test Error: 
 Accuracy: 58.5%, Avg loss: 1.884496 

Epoch 3
-------------------------------
loss: 1.927039  [   64/60000]
loss: 1.888283  [ 6464/60000]
loss: 1.767381  [12864/60000]
loss: 1.799533  [19264/60000]
loss: 1.712507  [25664/60000]
loss: 1.663066  [32064/600

# Save and Load the Model

PyTorch models store the learned parameters in an internal state dictionary, called `state_dict`, via the `torch.save` method

In [11]:
import torchvision.models as models

In [12]:
model = models.vgg16(weights='IMAGENET1K_V1')
torch.save(model.state_dict(), 'model_weights.pth')

To load model weights, we create an instance of the same model, then load the parameters using `load_state_dict()` method.

In [13]:
model = models.vgg16()
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1